In [9]:
#Base Python imports
import sys
import os

In [7]:
##Get the location of raw_pickles, assuming you run this notebook in xrb-sens-datashare/raw_pickles
import os
cur_data_root = os.getcwd()
print('Using raw_pickles data root:\n   {}'.format(cur_data_root))
#'/home/ajacobs/Reporoot/xrb-sens-datashare/raw_pickles'

Using raw_pickles data root:
   /home/ajacobs/Reporoot/xrb-sens-datashare/raw_pickles


In [10]:
#Derive analysis and kepler locations from root
package_root = '{}/../analysis_packages'.format(cur_data_root)
kepler_package_root = '{}/../kepler_python_packages'.format(cur_data_root)

#Insert kepler python packages into the python path
sys.path.insert(0, '{}/python_scripts'.format(kepler_package_root))

In [11]:
#Provide analysis, data abstraction, and reduction modules as well as modules
#that understand Kepler's data formats
#NOTE: This is very hacky and brittle. Should be improved in next development iteration.
import re
from pickle import dump, load
from glob import glob
from os.path import basename, join
from random import sample
from scipy.constants import physical_constants

#import grid_setup
#from model_analysis import ModelAnalysis
from lcdata import LCData
from kepdata import KepData
from kepdump import KepDump
from isoplot import IsoPlot
from nucplot import IsoPlot as NucPlot
import bdat
import ionmap
from isotope import el2z

With dependencies provided, reading up to the entire raw data set is as easy as
manipulating a nested Python dictionary full of performant numpy objects as
well as self-documenting labels/descriptions.

Below, we explore the dictionary at a high level

In [12]:
#Below is a reference example of leading in the pickle, doing some checks, and
#highlighting some of the detailed data available.

#Load in the raw data
#with open('4u1820ZJ_100x_grid_data.Nov18.pk', 'rb') as f:
with open('gs1826_10x_grid_data.Nov5.pk', 'rb') as f:
    grid_data = load(f)

gd = grid_data

#Do a basic check of the data
#summarize the grid properties
print("Grid label:       {}".format(gd['grid_label']))
print("Grid description: {}".format(gd['grid_desc']))
print("X:      {}".format(gd['x']))
print("Z(n14): {}".format(gd['z']))
print("Q_b:    {} MeV / u".format(gd['qb']))
print("Eddington fraction: {}".format(gd['eddf']))
print("xi: {}".format(gd['xi']))
print("Accretion Lum: {} erg / s".format(gd['acc_lum']))
print("Accretion Rate: {} M_sol / yr".format(gd['acc_rate']))
print("surface gravity, g: {} cm / s^2".format(gd['gee']))
print("# of variations (may be off by 1, see source): {}".format(len(gd['vary_list'])-1)) #-1 because we currently put fake variation in first to gen the baseline
print("# of models: {}".format(len(gd['model_data'])))
print("")

Grid label:       gs1826_10x
Grid description: The full set of 10x variations on the GS 1826 base
X:      0.7
Z(n14): 0.0075
Q_b:    0.025 MeV / u
Eddington fraction: 0.19
xi: 1.0
Accretion Lum: 5.055391275e+33 erg / s
Accretion Rate: 3.3250000000000004e-09 M_sol / yr
surface gravity, g: 2.654949024e+16 cm / s^2
# of variations (may be off by 1, see source): 2491
# of models: 2474



The model data is in `grid_data['model_data']` and is keyed based on the model's reaction variations.

For example, `grid_data['model_data']['dag0.1']` contains data for the baseline model (GS1826 in this case) with a variation of the d(alpha, gamma) reaction down by a factor of 10 (hence 0.1).

`grid_data['model_data']['o15ag10.0']` contains data for O15 (alpha, gamma) being varied up by a factor of 10.

Model labels generally follow this form.  Below, we print out a list of all model labels.  Note that on occasion special labels are made, like 'base'.

In [29]:
model_count = len(gd['model_data'])
print('There are a total of {} models in model_data\n'.format(model_count))
labels = (label for label in gd['model_data'].keys())

for label in labels:
    print(label)

There are a total of 2474 models in model_data

base
dag0.1
tag10.0
tag0.1
he3ag10.0
he3ag0.1
li6ag10.0
li6ag0.1
c14ag10.0
c14ag0.1
n14ag10.0
n14ag0.1
n15ag10.0
n15ag0.1
o14ag10.0
o14ag0.1
o15ag10.0
o15ag0.1
o16ag10.0
o16ag0.1
ni56ag10.0
ni56ag0.1
ni57ag10.0
ni57ag0.1
ni58ag10.0
ni58ag0.1
cu59ag10.0
cu60ag10.0
cu60ag0.1
cu61ag10.0
cu62ag10.0
cu62ag0.1
zn59ag10.0
zn59ag0.1
zn60ag10.0
zn60ag0.1
zn61ag0.1
zn62ag10.0
zn62ag0.1
zn63ag10.0
zn63ag0.1
ga63ag10.0
ga64ag10.0
ga65ag10.0
ga65ag0.1
ge61ag10.0
ge61ag0.1
ge62ag10.0
ge62ag0.1
ge63ag10.0
ge63ag0.1
ge64ag10.0
ge65ag10.0
ge66ag10.0
ge66ag0.1
as67ag10.0
as67ag0.1
as68ag10.0
as68ag0.1
se65ag10.0
se65ag0.1
se66ag10.0
se66ag0.1
se67ag10.0
se67ag0.1
se68ag10.0
se68ag0.1
se69ag10.0
se69ag0.1
tc93ag10.0
tc93ag0.1
tc94ag10.0
tc94ag0.1
ru94ag10.0
ru94ag0.1
ru95ag10.0
ru95ag0.1
rh95ag10.0
rh95ag0.1
rh96ag10.0
rh96ag0.1
rh97ag10.0
rh97ag0.1
pd96ag10.0
pd96ag0.1
pd97ag10.0
pd97ag0.1
pd98ag10.0
pd98ag0.1
pd99ag10.0
pd99ag0.1
ag97ag10.0
ag97ag0.1
ag98

Each model has associated data, as we see here

In [55]:
print(gd['model_data']['o15ag10.0'].keys())
print('\n')

#Overview and bulk data
for key in list(gd['model_data']['o15ag10.0'].keys())[0:5]:
    print('{} --> {}'.format(key, gd['model_data']['o15ag10.0'][key]))
key='kepler_label' #this is a simple short label used by kepler to keep track of the model
print('{} --> {}'.format(key, gd['model_data']['o15ag10.0'][key]))
    
    
    
#Raw full lightcurve data
print('raw_lc_time --> <numpy array of raw lightcurve time in seconds>')
print('raw_lc_lum --> <numpy array of raw lightcurve luminosity in erg/s>')
print('raw_lc_rad --> <numpy array of raw lightcurve photosphere radius in cm>')

#Burst data in which the above lightcurve is broken up into distinct bursts (`nbursts` tells you how many there are)
print('burst_times --> <a list of `nburst` numpys arrays containing that burst time, with t=0s being the peak time>')
print('burst_lums  --> <a list of `nburst` numpys arrays containing that burst luminosity L(t), with t coming from corresponding burst_times>')
print('alc_arr     --> <a list of [time, lum, rad] numpy arrays representing the average of all bursts>')

#Detailed profile (ash!) data is available through kepler's dump files:
print(gd['model_data']['o15ag10.0']['dump_data'].keys())

#More data is available for each dumpfile, but some in the pickles are still incomplete and only have simple metadata (ncyc, jblo, path, etc):
print(gd['model_data']['o15ag10.0']['dump_data']['gs18#32000'].keys())

dict_keys(['model_label', 'model_desc', 'grid_path', 'nbursts', 't_rec', 'fluence', 'raw_lc_time', 'raw_lc_lum', 'raw_lc_rad', 'burst_times', 'burst_lums', 'alc_arr', 'kepler_label', 'dump_data'])


model_label --> o15ag10.0
model_desc --> GS 1826 model with rxn o15ag scaled by 10.0
grid_path --> ['/mnt/home/jacob308/Research/Projects/XRB/Sensitivity/runs/gs1826', 'grid_10x', 'models', 'gs1826_1']
nbursts --> 38
t_rec --> (2.799741795439786, 0.8928910281437881)
kepler_label --> gs18
raw_lc_time --> <numpy array of raw lightcurve time in seconds>
raw_lc_lum --> <numpy array of raw lightcurve luminosity in erg/s>
raw_lc_rad --> <numpy array of raw lightcurve photosphere radius in cm>
burst_times --> <a list of `nburst` numpys arrays containing that burst time, with t=0s being the peak time>
burst_lums  --> <a list of `nburst` numpys arrays containing that burst luminosity L(t), with t coming from corresponding burst_times>
alc_arr     --> <a list of [time, lum, rad] numpy arrays represen